In [3]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;

use vawt::areofoil::Aerofoil;
use vawt::{VAWTSolver, Verbosity, VAWTSolution};

In [4]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [5]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;
let aerofoil: &'static Aerofoil = Box::leak(Box::new(aerofoil));
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(72)
    .verbosity(Verbosity::Silent);

In [6]:
let solution = solver.tsr(3.25).solve_with_beta(0.0);

let n = 72;
let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..2.0, 0f64..1f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}π", x))
        .draw()?;

    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.a(theta))), 
        &RED
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta / PI, solution.c_tan(theta))), 
        &BLUE
    ))?;
    chart.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta / PI, solution.thrust_error(theta))),
        &GREEN
    ))?;
    Ok(())
})


0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
 
 
0.0π
 
 
 
0.5π
 
 
 
1.0π
 
 
 
1.5π
 
 
 
2.0π
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="44,345 52,386 60,376 69,366 77,352 85,335 94,325 102,311 110,301 119,290 127,283 135,280 143,277 152,280 160,283 168,290 177,297 185,307 193,318 202,325 210,328 218,335 227,342 235,345 243,352 252,355 260,359 268,359 277,362 285,362 293,366 302,376 310,390 318,407 327,424 335,440 343,440 351,424 360,400 368,373 376,345 385,314 393,287 401,266 410,246 418,235 426,222 435,211 443,198 451,187 460,177 468,167 476,160 485,150 493,139 501,126 510,119 518,112 526,109 535,105 543,112 551,122 559,139 568,160 576,184 584,215 593,239 601,280 609,318 618,345 626,362 634,139 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="44,440 52,440 60,440 69,439 77,435 85,431 94,427 102,424 110,420 119,417 127,413 135,410 143,407 152,404 160,402 168,401 177,400 185,401 193,401 202,402 210,402 218,403 227,403 235,404 243,404 252,405 260,405 268,405 277,405 285,406 293,411 302,418 310,426 318,435 327,440 335,440 343,440 351,440 360,439 368,437 376,436 385,436 393,436 401,435 410,435 418,434 426,435 435,435 443,437 451,438 460,439 468,440 476,440 485,440 493,440 501,440 510,440 518,440 526,440 535,440 543,440 551,440 559,440 568,440 576,440 584,440 593,440 601,440 609,440 618,440 626,440 634,440 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="44,440 52,436 60,440 69,437 77,434 85,440 94,436 102,440 110,439 119,440 127,440 135,440 143,440 152,437 160,438 168,439 177,440 185,440 193,437 202,437 210,440 218,440 227,438 235,440 243,437 252,438 260,438 268,440 277,437 285,436 293,440 302,440 310,440 318,435 327,440 335,440 343,440 351,440 360,440 368,440 376,440 385,440 393,440 401,438 410,440 418,439 426,440 435,434 443,435 451,440 460,438 468,437 476,440 485,440 493,434 501,440 510,440 518,440 526,440 535,440 543,440 551,440 559,432 568,440 576,437 584,433 593,440 601,440 609,440 618,440 626,434 634,440 "/>

In [7]:
let tsr = Array::linspace(1.0,5.0,20);
let cps = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta(0.0).ct_cp().1
    })
);

In [8]:
evcxr_figure((640,480), |root| {    
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 0f64..1f64)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps.iter().cloned()), 
        &RED
    ))?;
    Ok(())
})

0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0